In [1]:
##################################################################################################################
############################## LLM ENGINEERING: Jordi Ortega - IA2 ###############################################
##################################################################################################################


# GenAI Project Lifecycle. Pasos aplicados en este notebook:
# 1. DefiniCión, modelo y adaptación (Entrenamos al modelo para que tenga conocimiento sobre salud mental)
# 2. Conclusiones

In [2]:
##################################################################################################################
############################## DEFINICIÓN, MODELO y ADAPTACIÓN ###################################################
##################################################################################################################

In [3]:
from huggingface_hub import login

# Usamos token de perfil propio para descarga del modelo en HF
login(token="hf_RDUAAOmxxxxxxxxxxxxxxxxxxsnCvKANYKB")

In [4]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires f

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargamos el modelo
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("JordiOrtega/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("JordiOrtega/distilgpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [8]:
# Descargamos el dataset y lo guardarmos en Drive
from datasets import load_dataset
import os

dataset_name = "Amod/mental_health_counseling_conversations"
dataset = load_dataset(dataset_name)

drive_dataset_path = "/content/drive/MyDrive/datasets"
os.makedirs(drive_dataset_path, exist_ok=True)
dataset.save_to_disk(drive_dataset_path)
print("Dataset guardado en Google Drive.")

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset guardado en Google Drive.


In [9]:
# Confirmamos la estructura del dataset
dataset['train'][0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [11]:
# Filtramos los registros en 'train' donde 'output' no es None ni vacío
dataset['train'] = dataset['train'].filter(lambda row: row['Response'] and row['Response'].strip() != "")

print(f"Tamaño después del filtro: {len(dataset['train'])}")


Filter:   0%|          | 0/3512 [00:00<?, ? examples/s]

Tamaño después del filtro: 3508


In [12]:
# Dividimos el dataset en train y test
dataset_train_test = dataset["train"].train_test_split(test_size=0.2)
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2806
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 702
    })
})

In [13]:
# Dividimos el dataset test en validación
dataset_val_test = dataset_train_test["test"].train_test_split(test_size=0.5)
dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
})

In [14]:
from datasets import DatasetDict

# Estructura del dataset preparada para el modelo
split_dataset = DatasetDict({
    "train" : dataset_train_test["train"],
    "val" : dataset_val_test["train"],
    "test" : dataset_val_test["test"]
})
split_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2806
    })
    val: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
})

In [15]:
# Creamos los dos templates de prompting, el de entreno y el de inferencia
SYSTEM_MESSAGE= "You are a personal assistant expert in mental health issues that responds concisely based on the given context in order to help people. You receive a context and you must provide a concrete and helpful answer."

PROMPT_TEMPLATE = """\
{bos_token}
{system_message}

### Context:
{Context}

### Response:
{Response}{eos_token}
"""

def create_prompt(sample):
  full_prompt = PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      Context = sample["Context"],
      Response = sample["Response"]
  )

  return {"text" : full_prompt}

In [16]:
INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}
{system_message}

### Context:
{Context}

### Response:
{eos_token}
"""

def create_prompt_and_response(sample):
  full_prompt = INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      Context = sample["Context"],
  )

  ground_truth = sample["Response"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

In [17]:
# Primer registro del dataset
split_dataset["train"][0]

{'Context': "I'm transgender, I know I am, but I've only told a few friends. I know I can't tell my family because of previous conversations we've had. They just wouldn't accept it.\r\nMy gender dysphoria is getting really difficult to deal with on my own. I need some strategies for dealing with it. What should I do?",
 'Response': 'This is a difficult situation to be in, as it sounds like you are feeling very isolated from both your family and your friends.\xa0\xa0 I don\'t know your age or gender so please excuse any incorrect assumptions about you being under 18 and use what is useful from the ideas if you are older.\xa0 One idea is to research online to find a therapist or a local clinic that has a therapist who is a "Gender Therapist" or a "Gender Specialist."\xa0 Most therapists who are transgender affirmative also have other specialties and do general therapy.\xa0 Though I don\'t think lying to your parents is a good idea, if you feel you truly can\'t talk to them about your gen

In [18]:
# Aplicamos la función al primer registro del dataset
create_prompt(split_dataset["train"][0])

{'text': '<|startoftext|>\nYou are a personal assistant expert in mental health issues that responds concisely based on the given context in order to help people. You receive a context and you must provide a concrete and helpful answer.\n\n### Context:\nI\'m transgender, I know I am, but I\'ve only told a few friends. I know I can\'t tell my family because of previous conversations we\'ve had. They just wouldn\'t accept it.\r\nMy gender dysphoria is getting really difficult to deal with on my own. I need some strategies for dealing with it. What should I do?\n\n### Response:\nThis is a difficult situation to be in, as it sounds like you are feeling very isolated from both your family and your friends.\xa0\xa0 I don\'t know your age or gender so please excuse any incorrect assumptions about you being under 18 and use what is useful from the ideas if you are older.\xa0 One idea is to research online to find a therapist or a local clinic that has a therapist who is a "Gender Therapist" or

In [19]:
# Pasamos por el template todo el dataset
split_dataset = split_dataset.map(create_prompt)

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/351 [00:00<?, ? examples/s]

Map:   0%|          | 0/351 [00:00<?, ? examples/s]

In [20]:
from transformers import pipeline, set_seed, GenerationConfig

# Creamos el pipeline para testear el modelo base (posteriormente lo haremos con el modelo finetuneado)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

set_seed(42)

def generate_sample(sample):
  prompt_package = create_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=100,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      repetition_penalty=1.2,
      eos_token_id=model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Question:")
  print(sample["Context"])
  print("---------------")
  print("Dataset Response")
  print(prompt_package["ground_truth"])
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))


Device set to use cuda:0


In [21]:
generate_sample(split_dataset["test"][15])
# Comprobamos que el modelo base no tiene expertise en salud mental.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Whenever I don't tell my friends or anyone what I did or stuff that's not really important, I feel terrible, like there’s a hole in my stomach. It only goes away when I hurt myself.
---------------
Dataset Response
Sorry to hear of your situation.Possibly you are overlooking that your own Self is someone worth talking to as well.None of us are ever really alone because we are always with ourselves.Self-talk is a major part of what guides our decisions and how we make sense of relationships and situations.    Even after telling people the stories or activities, hearing their comments, it is always within ourselves that we decide if the way we were received by these others, the comments we heard back, feel right and accurate to our lives.I hope the feeling of a hole in your stomach would decrease by enjoying your Self.    I am guessing that hurting yourself makes the stomach hole feel less bad because the physical pain you create in yourself distracts you from f

In [22]:
from transformers import TrainingArguments

# Definimos los argumentos del entreno
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=8,
 gradient_checkpointing =True,
 max_grad_norm= 1,
 max_steps=2000,
 learning_rate=1e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir="finetuned_mental_health_distilgpt2_v3",
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 evaluation_strategy="steps",
 eval_steps=100,
 warmup_ratio=0.1,
 report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
from trl import SFTTrainer

# Facilitamos la configuración a SFTTrainer (supervised finetuning)
trainer = SFTTrainer(
 model,
 train_dataset=split_dataset["train"],
 eval_dataset=split_dataset["val"],
 tokenizer=tokenizer,
 args=training_args
)

<ipython-input-23-b6571abee755>:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/351 [00:00<?, ? examples/s]

In [24]:
# Entrenamos
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
100,2.805700,2.710501
200,2.569600,2.538497
300,2.366700,2.426709
400,2.291100,2.342336
500,2.165100,2.276367
600,2.085200,2.211037
700,1.992400,2.173107
800,1.865500,2.128229
900,1.890600,2.093204
1000,1.761000,2.073242


TrainOutput(global_step=2000, training_loss=1.9401747288703919, metrics={'train_runtime': 7560.2703, 'train_samples_per_second': 8.465, 'train_steps_per_second': 0.265, 'total_flos': 8656663360831488.0, 'train_loss': 1.9401747288703919, 'epoch': 22.729344729344728})

In [25]:
trainer.save_model()

In [26]:
distilbert_mental_health_distilgpt2_finetuned = AutoModelForCausalLM.from_pretrained("finetuned_mental_health_distilgpt2_v3")

In [27]:
generator = pipeline('text-generation', model=distilbert_mental_health_distilgpt2_finetuned, tokenizer=tokenizer)

Device set to use cuda:0


In [28]:
# Testeamos ahora sí con el modelo finetuneado.
generate_sample(split_dataset["test"][20])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
I'm  very depressed. How do I find someone to talk to?
---------------
Dataset Response
Hang in there! Feeling depressed sucks but talking with someone can really help - so its awesome that you're already trying to figure out how to find that person. Tons of good suggestions already but to add my two cents - you can find therapists in your area and read a little bit about them to find someone you like the sound of via this website, and also via www.psychologytoday.com; if you have health insurance your insurer might be able to make recommendations as to therapists covered by your plan, or if you happen to be in school, the school itself should be able to connect you with either their own counselor or a center that they recommend nearby. Finding someone can take a few calls or a bit of research so if you need some immediate support, or find yourself having thoughts of hurting yourself, you can also call the great folks who run the Suicide crisis line https://su

In [29]:
# Conclusiones:
# Se ha repetido el ejercicio de '2_JordiOrtega_llm_engineering_finetuned_mental_health_v2.ipynb' pero con unos argumentos más exigente y el resultado es 
# mucho más satisfactorio. El modelo responde de forma coherente y desmuestra un conocimiento sobre la materia entrenada.

# En otras versiones se ha probado con distintos parámetros tanto en la configuración del prompt (temperature, penalty y tokenizer),
# como en el system message (...and never leave sentences unfinished or incomplete in the output) 
# como en los training arguments pero no se ha conseguido que el modelo conteste sin dejar frases sin terminar. (*** Se agradece feedback en especial sobre este punto) 